## 有名なモデルを実装

In [1]:
import torch
from torch.nn import functional as F
from torch import nn, optim
from torch.utils.data import DataLoader, Subset
import torchvision
from torchvision import models, transforms
from torchvision.models.vgg import VGG11_Weights

%load_ext autoreload
%autoreload 2
import utils

### LeNet

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, X):
        X = F.max_pool2d(F.relu(self.conv1(X)), 2)
        X = F.max_pool2d(F.relu(self.conv2(X)), 2)
        X = self.flatten(X)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        return X

In [3]:
lenet = LeNet()
X = torch.randn((128, 1, 32, 32))
out = lenet(X)
out.shape

torch.Size([128, 10])

### 既存のモデルの使用

In [4]:
model = models.vgg11(weights=VGG11_Weights.IMAGENET1K_V1)
model.classifier

Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth
100%|██████████| 507M/507M [00:55<00:00, 9.51MB/s] 


Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [6]:
# データ準備
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    # CIFAR10用
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
train_dataset = torchvision.datasets.CIFAR10('./cifar10_data', train=True, download=True, transform=transform)
val_dataset = torchvision.datasets.CIFAR10('./cifar10_data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
# データ量が多くて学習できない場合はサブデータを使用する
train_dataset_sub = Subset(train_dataset, range(500))
val_dataset_sub = Subset(val_dataset, range(250))
train_loader = DataLoader(train_dataset_sub, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset_sub, batch_size=16, num_workers=4)

In [7]:
# 最終層の出力を10クラス分類用にする
model.classifier[-1] = nn.Linear(4096, 10)

In [8]:
# GPUで学習する場合は以下のように.to(device)でGPUにモデルを移動させる
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model.to(device)
opt = optim.SGD(model.parameters(), lr=0.03)

In [ ]:
import time
start = time.time()
train_losses, val_losses, val_accuracies = utils.learn(model, train_loader, val_loader, opt, F.cross_entropy, 3)
end = time.time()
print(end - start)

Training:   0%|          | 0/32 [00:00<?, ?it/s]